# Benchmarking skforecast Recursive Forecasters

This notebook benchmarks the performance (velocity) of the `skforecast` in its different versions and keeps track of the results.

**Notes**

+ In version `0.15.0` the binning of residuals was introduced in multi-series forecasters. This explains the increase in the time taken to fit the model.
+ Since version `0.17.0`, the `RecursiveMultiSeriesForecaster` only accepts as input a long format DataFrame with a MultiIndex or a dictionary of series. Wide format DataFrames where each column is a different time series are no longer supported. If input data is a pandas dataframe with with MultiIndex, it is internally converted to a dictionary of series what increases notably the computation time.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import sklearn
import joblib
import skforecast
import platform
import psutil
import plotly.express as px
from benchmarks.utils import plot_benchmark_results

In [3]:
print(f"Python version: {platform.python_version()}")
print(f"skforecast version: {skforecast.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"Computer network name: {platform.node()}")
print(f"Processor type: {platform.processor()}")
print(f"Platform type: {platform.platform()}")
print(f"Operating system: {platform.system()}")
print(f"Operating system release: {platform.release()}")
print(f"Operating system version: {platform.version()}")
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.12.11
skforecast version: 0.17.0
scikit-learn version: 1.6.1
pandas version: 2.3.1
numpy version: 2.1.3
Computer network name: ITES015-NB0029
Processor type: Intel64 Family 6 Model 141 Stepping 1, GenuineIntel
Platform type: Windows-11-10.0.26100-SP0
Operating system: Windows
Operating system release: 11
Operating system version: 10.0.26100
Number of physical cores: 8
Number of logical cores: 16


In [4]:
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="'force_all_finite' was renamed to 'ensure_all_finite'"
)

# ForecasterRecursive

In [6]:
# Plot results
# ==============================================================================
display_df = False
selected_date = None
# 'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'
# 'Windows-11-10.0.26100-SP0'
selected_platform = None
python_version = None

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark = results_benchmark_all.query("forecaster_name in ['ForecasterRecursive', 'ForecasterAutoreg']")
results_benchmark = results_benchmark.query("regressor_name == 'DummyRegressor'")
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'")
    if selected_date:
        df = df[df['datetime'].dt.date == pd.to_datetime(selected_date).date()]
    if selected_platform:
        df = df[df['platform'] == selected_platform]
    if python_version:
        df = df[df['python_version'] == python_version]
    if df.empty:
        print(f"No results found for function '{function_name}' with the selected filters.")
    if display_df:
        display(df.tail(3))
    plot_benchmark_results(df.copy(), function_name, add_median=True, add_mean=True)

# Summary of historical results

In [6]:
selected_date = None
# 'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'
# 'Windows-10-10.0.19045-SP0'
selected_platform = None

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark_all = results_benchmark_all.query("regressor_name == 'DummyRegressor'")
results_benchmark_all['function_name'] = results_benchmark_all['function_name'].str.split('_', n=1).str[1]
if selected_date:
    results_benchmark_all = results_benchmark_all.query("datetime.dt.date == @selected_date")
if selected_platform:
    results_benchmark_all = results_benchmark_all.query("platform == @selected_platform")
results_benchmark_all = results_benchmark_all.groupby(['forecaster_name', 'skforecast_version', 'function_name'])['run_time_avg'].agg('median').reset_index()
results_benchmark_all = results_benchmark_all.sort_values(by=['function_name'])

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterRecursiveMultiSeries', 'ForecasterAutoregMultiSeries']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='MultiSeries Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterRecursive', 'ForecasterAutoreg']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='Single-Series Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()
